In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Evaluate and Compare Gen AI Model Settings

* [Evaluate and Compare Gen AI Model Settings | Gen AI Evaluation SDK Tutorial](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluate_and_compare_gemini_model_settings.ipynb#scrollTo=JAPoU8Sm5E6e)

In [1]:
# @title Install Vertex AI Python SDK
%pip install --upgrade --user --quiet google-cloud-aiplatform[evaluation]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 16.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# @title Define constants

PROJECT_ID="ai-hangsik" # @param {type:"string"}
LOCATION="us-central1" # @param {type:"string"}

In [3]:

# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

In [4]:
# @title Initialize Vertex AI SDK
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries

In [5]:
# General
import logging
import random
import string
import warnings

from IPython.display import HTML, Markdown, display
import pandas as pd

# Main
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
)

### Library settings

In [ ]:
# logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
# warnings.filterwarnings("ignore")

### Helper Functions

In [6]:
def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specified length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown("### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<h2>{col}:</h2> <div style='{style}'>{row[col]}</div>"))
        display(HTML("<hr>"))

## Define an Evaluation Dataset

In [7]:
instruction = "Summarize the following article: \n"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]

eval_dataset = pd.DataFrame(
    {
        "prompt": [instruction + item for item in context],
        "reference": reference,
    }
)

### Evaluate Gemini Model with different Generation Settings

In [8]:
experiment_name = "eval-model-generation-settings"  # @param {type:"string"}
summarization_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.TEXT_QUALITY,
        MetricPromptTemplateExamples.Pointwise.FLUENCY,
        MetricPromptTemplateExamples.Pointwise.SAFETY,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
    ],
    experiment=experiment_name,
)

In [10]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

generation_config_1 = GenerationConfig(
    max_output_tokens=64,
    temperature=0.1,
)
gemini_1 = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=generation_config_1,
    safety_settings=safety_settings,
)

generation_config_2 = GenerationConfig(
    max_output_tokens=128,
    temperature=0.3,
)
gemini_2 = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=generation_config_2,
    safety_settings=safety_settings,
)

generation_config_3 = GenerationConfig(
    max_output_tokens=256,
    temperature=0.8,
)
gemini_3 = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=generation_config_3,
    safety_settings=safety_settings,
)

In [11]:
models = {
    "gemini-setting-1": gemini_1,
    "gemini-setting-2": gemini_2,
    "gemini-setting-3": gemini_3,
}

In [12]:
eval_results = []
run_id = generate_uuid()

for model_name, model in models.items():
    experiment_run_name = f"eval-{model_name}-{run_id}"

    eval_result = summarization_eval_task.evaluate(
        model=model,
        experiment_run_name=experiment_run_name,
    )

    eval_results.append(
        (f"Model {model_name}", eval_result.summary_metrics, eval_result.metrics_table)
    )

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/eval-model-generation-settings-eval-gemini-setting-1-445ra305 to Experiment: eval-model-generation-settings


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'model_name': 'publishers/google/models/gemini-1.5-pro', 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.34it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 2.1571322839999993 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 20/20 [00:23<00:00,  1.15s/it]
INFO:vertexai.evaluation._evaluation:All 20 metric requests are successfully computed.
INFO:vertexai.evaluation.

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/eval-model-generation-settings-eval-gemini-setting-2-445ra305 to Experiment: eval-model-generation-settings


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'model_name': 'publishers/google/models/gemini-1.5-pro', 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.05it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 2.457835842999998 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 20/20 [00:23<00:00,  1.15s/it]
INFO:vertexai.evaluation._evaluation:All 20 metric requests are successfully computed.
INFO:vertexai.evaluation._

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/eval-model-generation-settings-eval-gemini-setting-3-445ra305 to Experiment: eval-model-generation-settings


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'model_name': 'publishers/google/models/gemini-1.5-pro', 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.15it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference took: 2.3372004190000553 seconds.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 20/20 [00:22<00:00,  1.14s/it]
INFO:vertexai.evaluation._evaluation:All 20 metric requests are successfully computed.
INFO:vertexai.evaluation.

### Evaluation Results

In [13]:
for eval_result in eval_results:
    display_eval_report(eval_result)

## Model gemini-setting-1

### Summary Metrics

,row_count,text_quality/mean,text_quality/std,fluency/mean,fluency/std,safety/mean,safety/std,verbosity/mean,verbosity/std
0,5.0,4.2,1.095445,3.8,1.643168,1.0,0.0,0.0,1.0


### Report Metrics

,prompt,reference,response,text_quality/explanation,text_quality/score,fluency/explanation,fluency/score,safety/explanation,safety/score,verbosity/explanation,verbosity/score
0,Summarize the following article: \nTo make a c...,The process of making spaghetti carbonara invo...,This article provides a concise method for mak...,"Coherence: The response is mostly coherent, pr...",3.0,STEP 1: Assess grammar correctness: The respon...,2.0,"The response is safe, containing no harmful or...",1.0,STEP 1: Assess completeness: The response prov...,-1.0
1,Summarize the following article: \nPreparing a...,Preparing risotto entails sautéing onions and ...,"Making perfect risotto is a labor of love, req...","Coherence: The response is mostly coherent, pr...",3.0,STEP 1: Assess grammar correctness: The respon...,2.0,"The response is safe, as it contains none of t...",1.0,STEP 1: Completeness: The response covers all ...,-1.0
2,Summarize the following article: \nFor a flavo...,Grilling a flavorful steak involves seasoning ...,"To grill a flavorful steak, choose a well-marb...",Coherence: The response presents the steps in ...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,"The response is safe, as it does not contain a...",1.0,STEP 1: Assess completeness: The response incl...,0.0
3,Summarize the following article: \nCreating a ...,Creating homemade tomato soup includes sautéin...,This article provides a simple recipe for crea...,Coherence: The response presents the steps in ...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe as it accurately summariz...,1.0,STEP 1: Assess completeness: The response incl...,1.0
4,Summarize the following article: \nTo bake a d...,Baking a decadent chocolate cake requires crea...,This article provides a concise guide to bakin...,Coherence: The response presents the ideas in ...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe as it does not contain an...,1.0,STEP 1: The response covers the main points of...,1.0


...

## Model gemini-setting-3

### Summary Metrics

,row_count,text_quality/mean,text_quality/std,fluency/mean,fluency/std,safety/mean,safety/std,verbosity/mean,verbosity/std
0,5.0,5.0,0.0,5.0,0.0,1.0,0.0,0.6,0.894427


### Report Metrics

,prompt,reference,response,text_quality/explanation,text_quality/score,fluency/explanation,fluency/score,safety/explanation,safety/score,verbosity/explanation,verbosity/score
0,Summarize the following article: \nTo make a c...,The process of making spaghetti carbonara invo...,This article provides a concise recipe for cla...,Coherence: The response presents the informati...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe as it provides a summary ...,1.0,STEP 1: Completeness: The response covers all ...,1.0
1,Summarize the following article: \nPreparing a...,Preparing risotto entails sautéing onions and ...,Making perfect risotto is a labor of love that...,Coherence: The response presents the steps inv...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe. It contains a summary of...,1.0,STEP 1 (Completeness): The response includes a...,1.0
2,Summarize the following article: \nFor a flavo...,Grilling a flavorful steak involves seasoning ...,"To grill a flavorful steak, choose a well-marb...",Coherence: The response is coherent and easy t...,5.0,STEP 1: The response is grammatically correct ...,5.0,The response provides a concise and accurate s...,1.0,STEP 1: Completeness: The response covers all ...,-1.0
3,Summarize the following article: \nCreating a ...,Creating homemade tomato soup includes sautéin...,This article provides a simple recipe for crea...,STEP 1: Assessment:\nCoherence: The response i...,5.0,STEP 1: The response is grammatically correct ...,5.0,"The response is safe, as it contains a simple ...",1.0,STEP 1: The response provides all the necessar...,1.0
4,Summarize the following article: \nTo bake a d...,Baking a decadent chocolate cake requires crea...,This article provides a concise guide to bakin...,Coherence: The response presents the ideas in ...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe. It provides a summary of...,1.0,STEP 1: Completeness: The response covers all ...,1.0


In [14]:
for eval_result in eval_results:
    display_explanations(eval_result[2], metrics=["fluency"])

In [15]:
summarization_eval_task.display_runs()

,experiment_name,run_name,run_type,state,param.HARM_CATEGORY_HATE_SPEECH,param.HARM_CATEGORY_HARASSMENT,param.HARM_CATEGORY_DANGEROUS_CONTENT,param.HARM_CATEGORY_UNSPECIFIED,param.HARM_CATEGORY_SEXUALLY_EXPLICIT,param.model_name,metric.safety/std,metric.row_count,metric.fluency/std,metric.verbosity/mean,metric.text_quality/std,metric.verbosity/std,metric.fluency/mean,metric.safety/mean,metric.text_quality/mean
0,eval-model-generation-settings,eval-gemini-setting-3-445ra305,system.ExperimentRun,COMPLETE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,publishers/google/models/gemini-1.5-pro,0.0,5.0,0.000000,0.6,0.000000,0.894427,5.0,1.0,5.0
1,eval-model-generation-settings,eval-gemini-setting-2-445ra305,system.ExperimentRun,COMPLETE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,publishers/google/models/gemini-1.5-pro,0.0,5.0,0.000000,0.8,0.000000,0.447214,5.0,1.0,5.0
2,eval-model-generation-settings,eval-gemini-setting-1-445ra305,system.ExperimentRun,COMPLETE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,BLOCK_NONE,publishers/google/models/gemini-1.5-pro,0.0,5.0,1.643168,0.0,1.095445,1.000000,3.8,1.0,4.2
